In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeWorkingHRDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "fitbit/17-9-24/takeout-20240917T195619Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)


/home/chowder/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

In [4]:
for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")

read in 7000000 rows from 39 files


In [5]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
else:
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]

2023-07-30


In [6]:
#takes like 50 seconds to make 16 million rows
import json
import pytz
from datetime import datetime

def to_iso(s):
    return '20' + s[6:8] + '-' + s[0:2] + '-' + s[3:5] + s[8:]

samplesCount = 0
samplesList = []
for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        sampleDT = datetime.fromisoformat(to_iso(row["dateTime"]))
        samplesList.append([sampleDT, 
                            row["value"]["confidence"], 
                            row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 1_000_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 1000000 samples so far
added 2000000 samples so far
added 3000000 samples so far
added 4000000 samples so far
added 5000000 samples so far
added 6000000 samples so far
added 7000000 samples so far
added 8000000 samples so far
added 9000000 samples so far
added 9072969 samples


In [7]:
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf.index = fitbitHRdf.index.tz_localize('UTC')

fitbitHRdf.index = fitbitHRdf.index.tz_convert("US/Arizona")

fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [8]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [9]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf.iloc[5000000:12000000], True)

a million rows took up 5.471787452697754 MB
putting 182755 rows per file
no files found making new ones
saving to 39 number of files
saving rows 0 to 182754
to a file named 2021-08-12T213906-0700_2021-08-29T171306-0700.parquet.gzip
saving rows 182755 to 365509
to a file named 2021-08-29T171311-0700_2021-09-15T131804-0700.parquet.gzip
saving rows 365510 to 548264
to a file named 2021-09-15T131809-0700_2021-10-02T094743-0700.parquet.gzip
saving rows 548265 to 731019
to a file named 2021-10-02T094748-0700_2021-10-19T023436-0700.parquet.gzip
saving rows 731020 to 913774
to a file named 2021-10-19T023446-0700_2021-11-05T225808-0700.parquet.gzip
saving rows 913775 to 1096529
to a file named 2021-11-05T225813-0700_2021-11-25T012018-0700.parquet.gzip
saving rows 1096530 to 1279284
to a file named 2021-11-25T012023-0700_2021-12-16T112624-0700.parquet.gzip
saving rows 1279285 to 1462039
to a file named 2021-12-16T112629-0700_2022-01-05T052513-0700.parquet.gzip
saving rows 1462040 to 1644794
to a

In [10]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

a million rows took up 5.472912788391113 MB
putting 182718 rows per file
saving rows 0 to 182718
to a file named 2020-05-13T100219-0700_2020-05-29T204833-0700.parquet.gzip
saving rows 182718 to 365436
to a file named 2020-05-29T204833-0700_2020-06-15T175149-0700.parquet.gzip
saving rows 365436 to 548154
to a file named 2020-06-15T175149-0700_2020-07-02T173851-0700.parquet.gzip
saving rows 548154 to 730872
to a file named 2020-07-02T173851-0700_2020-07-19T103931-0700.parquet.gzip
saving rows 730872 to 913590
to a file named 2020-07-19T103931-0700_2020-08-05T033638-0700.parquet.gzip
saving rows 913590 to 1096308
to a file named 2020-08-05T033638-0700_2020-08-22T011641-0700.parquet.gzip
saving rows 1096308 to 1279026
to a file named 2020-08-22T011641-0700_2020-09-07T230719-0700.parquet.gzip
saving rows 1279026 to 1461744
to a file named 2020-09-07T230719-0700_2020-09-25T101037-0700.parquet.gzip
saving rows 1461744 to 1644462
to a file named 2020-09-25T101037-0700_2020-10-12T015335-0700.pa

In [11]:
"2023-06-27T010355-0700_2023-07-10T061055-0700.parquet.gzip".replace('.', '_').split('_')

['2023-06-27T010355-0700', '2023-07-10T061055-0700', 'parquet', 'gzip']